In [1]:
TEST_BAM = '/gscmnt/gc2686/rna_editing/data/cptac3/ccrcc/C3L-00004.T.bam'
TEST_POSITIONS = 'positions.bed'

In [2]:
import sys
sys.path.insert(0, '../rna-editing-downstream')

In [12]:
import sortedcontainers
import bam_utils
import subprocess
import re

In [14]:
def get_chrom_start_cigar_seq_read_tups(input_bam_fp, positions_fp, max_depth=10000):
    print('first')
    tool_args = ['samtools', 'view',
            '-L', positions_fp,
            input_bam_fp]
    print(tool_args)
    ps_1 = subprocess.Popen(tool_args, stdout=subprocess.PIPE)
    output = subprocess.check_output(('cut', '-f', '3,4,6,10'), stdin=ps_1.stdout).decode('utf-8')
    ps_1.wait()

    print('here')

    read_tups = []
    for line in output.split('\n'):
        if line:
            pieces = line.split('\t')
            chrom, pos, cigar, seq = pieces[0], pieces[1], pieces[2], pieces[3]
            read_tups.append((chrom, pos, cigar, seq))

    return read_tups

In [10]:
def join_segments(start, stop, seq, segment_first, segment_last):
    segment = {
        'start': start,
        'stop': stop,
        'sequence': seq
    }
    if start <= segment_first['start']:
        segment['start'] = start
    else:
        segment['start'] = segment_first['start']
#         if stop <= segment_first['stop']:
        segment['sequence'] = segment_first['sequence'][:start - segment_first['start']] + segment['sequence']
        
    if stop >= segment_last['stop']:
        segment['stop'] = stop    
    else:
        segment['stop'] = segment_last['stop']
#         if 
        cut_index = len(segment_last['sequence']) - (segment_last['stop'] - stop)
        segment['sequence'] = segment['sequence'] + segment_last['sequence'][cut_index:]
        
    return segment

class ReferenceCollection(object):
    def __init__(self):
        self.chrom_to_sorted_segments = {}
    
    def put_sequence(self, chrom, start, stop, sequence):
        start, stop = int(start), int(stop)
        if chrom not in self.chrom_to_sorted_segments:
            d = sortedcontainers.SortedDict()
            d[start] = {
                'start': start,
                'stop': stop,
                'sequence': sequence
            }
            
            self.chrom_to_sorted_segments[chrom] = d
        else:
            d = self.chrom_to_sorted_segments[chrom]
            i, j = d.bisect_left(start), d.bisect_right(stop)
            
            # if start == stop then we need to add a new segment
            # otherwise we need to combine segments
            if i == j:
                print('here')
                d[start] = {
                    'start': start,
                    'stop': stop,
                    'sequence': sequence
                }
            else:
                segments = list(d.items())[i:j]
                segment_first_start, segment_first = segments[0]
                segment_last_start, segment_last = segments[-1]
                
                segment = join_segments(start, stop, seq,
                        segment_first, segment_last)
                
                # remove old segments
                for k, _ in segments:
                    d.pop(k)
                
                d[segment['start']] = segment
                
    def get_sequence(self, chrom, start, stop):
        pass
    
#     def __str__(self):
#         return self.chrom_to_sorted_segments

In [8]:
TEST_BAM, TEST_POSITIONS

('/gscmnt/gc2686/rna_editing/data/cptac3/ccrcc/C3L-00004.T.bam',
 'positions.bed')

In [15]:
get_chrom_start_cigar_seq_read_tups(TEST_BAM, TEST_POSITIONS)

first
['samtools', 'view', '-L', 'positions.bed', '/gscmnt/gc2686/rna_editing/data/cptac3/ccrcc/C3L-00004.T.bam']
here


[]

In [ ]:
!samtools view -L positions.bed /gscmnt/g

In [5]:
read_tups = bam_utils.get_chrom_start_cigar_seq_read_tups(TEST_BAM, TEST_POSITIONS)

In [6]:
read_tups

[]

In [ ]:
# sequences = [
#     ('chr1', 11, 100, 'abcde' * 18),
#     ('chr1', 105, 110, 'helllo'),
#     ('chr1', 5, 15, 'hellloabcde'),
#     ('chr1', 110, 115, 'abcdehello'),
# ]
rc = ReferenceCollection()

for chrom, start, cigar, seq in read_tups:
    stop = start + len(seq)
    print(chrom, start, stop, seq)
    rc.put_sequence(chrom, start, stop, seq)

In [ ]:
rc.chrom_to_sorted_segments

In [ ]:
d = sortedcontainers.SortedDict({
    1: 'hello',
    10: 'world',
    100: '!',
    2: 'poop'
})

In [ ]:
ks = d.keys()

In [ ]:
i = d.bisect_left(11)
j = d.bisect_right(999)
i, j

In [ ]:
d.values()[i]

In [ ]:
for k, s in list(d.items())[i:j]:
    print(k, s)